In [23]:
import numpy as np
from PIL import Image
from scipy.signal import fftconvolve as fft
from scipy.stats import norm

In [24]:
def make_gaussian(sigma):
    """
    Build a 2D Gaussian filter.

    With n = 3*sigma, this returns a (2n+1) x (2n+1) array where each entry
    approximates the integral of the continuous Gaussian over a single pixel.
    The filter is normalized so that its total sum is 1.

    Parameters
    ----------
    sigma : float
        Standard deviation of the Gaussian.

    Returns
    -------
    G : ndarray
        A normalized 2D Gaussian filter.
    """

    # n = 3*sigma
    n = int(3 * sigma)
    N = 2 * n + 1

    # 1D Gaussian
    def g(x):
        return norm.pdf(x, 0, sigma)

    # Riemann sum
    m = 1000
    M = N * m

    # N intervals of length 1
    x = np.linspace(-n - 0.5, n + 0.5, num=M, endpoint=False)
    fx = g(x)

    # Tensor Product
    A = np.kron(np.eye(N), np.ones((m, 1)))                    

    # Integrate
    I_vec = (1.0 / m) * (fx @ A)

    # 2D Gaussian
    G = np.outer(I_vec, I_vec)

    # Normalize
    G /= G.sum()

    return G

In [25]:
def g_sharpen(im, sigma, alpha):
    """
    Sharpen an image using Gaussian. The image is blurred with a Gaussian filter,
    the high-frequency part (im - blur) is scaled by alpha, 
    and the result is added back to the original.
    Five images are returned so the full process can be seen.

    Parameters
    ----------
    im : PIL.Image.Image
        Input RGB image.
    sigma : float
        Standard deviation used for Gaussian.
    alpha : float
        Strength of the sharpening.

    Returns
    -------
    images : list of PIL Images
        [original, blurred, high_freq, scaled_high_freq, sharpened]
    """
    
    # convert to float array
    im_arr = np.asarray(im, dtype=float)

    # Gaussian
    G = make_gaussian(sigma)

    # run Gaussian using FFT
    gim_array = fft(im_arr, G[:, :, None], mode='same')

    # high-frequency component
    hf = im_arr - gim_array

    # scaled high-frequency
    scaled = alpha * hf

    # sharpened
    sharp = im_arr + scaled

    # convert to images 
    original = im
    gim = Image.fromarray(np.clip(gim_array, 0, 255).astype(np.uint8))
    hf_img = Image.fromarray(np.clip(hf, 0, 255).astype(np.uint8))
    scaled_img = Image.fromarray(np.clip(scaled, 0, 255).astype(np.uint8))
    sharp_img = Image.fromarray(np.clip(sharp, 0, 255).astype(np.uint8))

    return [original, gim, hf_img, scaled_img, sharp_img]

In [26]:
def log_sharpen(im, sigma, alpha):
    """
    Sharpen an image using the Laplacian of Gaussian.
    The image is first blurred with a Gaussian filter, then a Laplacian
    is applied to highlight edges. The Laplacian is
    scaled by alpha and added back to the original image. The five
    intermediate images are returned.

    Parameters
    ----------
    im : PIL.Image.Image
        Input RGB image.
    sigma : float
        Standard deviation used for Gaussian.
    alpha : float
        Strength of the LoG sharpening.

    Returns
    -------
    images : list of PIL Images
        [original, blurred, log_response, scaled_log, sharpened]
    """
    # image as float array
    im_arr = np.asarray(im, dtype=float)

    # Gaussian
    G = make_gaussian(sigma)
    gim_array = fft(im_arr, G[:, :, None], mode='same')

    # Laplacian
    lap = np.array([[-1, -1, -1],
                    [-1,  8, -1],
                    [-1, -1, -1]], dtype=float)

    # Laplacian of Gaussian
    log_array = fft(gim_array, lap[:, :, None], mode='same')

    # scaled LoG
    scaled_array = alpha * log_array

    # sharpened image
    sharp_array = im_arr + scaled_array

    # convert to images
    original = im
    gim = Image.fromarray(np.clip(gim_array,    0, 255).astype(np.uint8))
    log_img = Image.fromarray(np.clip(log_array, 0, 255).astype(np.uint8))
    scaled_img = Image.fromarray(np.clip(scaled_array, 0, 255).astype(np.uint8))
    sharp_img = Image.fromarray(np.clip(sharp_array, 0, 255).astype(np.uint8))

    return [original, gim, log_img, scaled_img, sharp_img]

In [27]:
im1 = Image.open("leaves.png").convert("RGB")
im2 = Image.open("cassette.png").convert("RGB")
out1 = g_sharpen(im1, sigma=3, alpha=2.0)
out2 = log_sharpen(im2, sigma=3, alpha=1.0)

In [28]:
images1 = [
    "im1_original.png",
    "im1_gblur.png",
    "im1_highfreq.png",
    "im1_scaledhf.png",
    "im1_sharpened.png"
]

for name, im in zip(images1, out1):
    im.save(name)

images2 = [ 
    "im2_original.png",
    "im2_gblur.png",
    "im2_log.png",
    "im2_scaledlog.png",
    "im2_sharpened.png"
]

for name, im in zip(images2, out2):
    im.save(name)